### 제목
- 웹 페이지를 다운로드하기 위한 라이브러리
- https://www.dataquest.io/blog/web-scraping-python-using-beautiful-soup/

In [2]:
import requests

page = requests.get("https://dataquestio.github.io/web-scraping-pages/simple.html")
page

<Response [200]>

In [3]:
#status_code  의미
# 2xx: 성공
# 4xx: 클라이언트 에러
# 5xx: 서버 에러
page.status_code

200

In [4]:
# binary response content
page.content

b'<!DOCTYPE html>\n<html>\n    <head>\n        <title>A simple example page</title>\n    </head>\n    <body>\n        <p>Here is some simple content for this page.</p>\n    </body>\n</html>'

### BeautifulSoup 라이브러리

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, "html.parser")  # parser : 구문 분석기

In [7]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


In [8]:
list(soup.children)

['html',
 '\n',
 <html>
 <head>
 <title>A simple example page</title>
 </head>
 <body>
 <p>Here is some simple content for this page.</p>
 </body>
 </html>]

In [9]:
# Doctype : 문서 타입에 대한 정보
# NavigableString : HTML 문서에서 찾아낸 텍스트
# Tag: 중첩(nasted) 태그
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]

In [10]:
# html 태그 안의 자식에 대한 결과
html = list(soup.children)[2]
list(html.children)

['\n',
 <head>
 <title>A simple example page</title>
 </head>,
 '\n',
 <body>
 <p>Here is some simple content for this page.</p>
 </body>,
 '\n']

In [11]:
# body 태그 안 자식에 대한 결과
body = list(html.children)[3]
list(body.children)

['\n', <p>Here is some simple content for this page.</p>, '\n']

In [12]:
# p 태그 안 자식에 대한 결과
p = list(body.children)[1]
p.get_text()

'Here is some simple content for this page.'

### 태그명으로 태그 검색

In [13]:
soup = BeautifulSoup(page.content, "html.parser")
soup.find_all('p')

[<p>Here is some simple content for this page.</p>]

In [14]:
# find_all은 list를 리턴함
# 따라서 첫 번째 텍스트를 열기 위해 다음과 같이 함
soup.find_all('p')[0].get_text() 

'Here is some simple content for this page.'

In [15]:
# 첫 번째 인스턴스만 찾기 원할 경우 find 이용
soup.find('p')

<p>Here is some simple content for this page.</p>

### class와 id로 태그 검색

In [21]:
page = requests.get("https://dataquestio.github.io/web-scraping-pages/ids_and_classes.html")
soup = BeautifulSoup(page.content, 'html.parser')
soup

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                First paragraph.
            </p>
<p class="inner-text">
                Second paragraph.
            </p>
</div>
<p class="outer-text first-item" id="second">
<b>
                First outer paragraph.
            </b>
</p>
<p class="outer-text">
<b>
                Second outer paragraph.
            </b>
</p>
</body>
</html>

In [22]:
soup.find_all('p', class_='outer-text')

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>,
 <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [23]:
soup.find_all(class_="outer-text")

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>,
 <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [24]:
soup.find_all(id="first")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>]

### CSS 선택자
- p a — finds all a tags inside of a p tag.
- body p a — finds all a tags inside of a p tag inside of a body tag.
- html body — finds all body tags inside of an html tag.
- p.outer-text — finds all p tags with a class of outer-text.
- p#first — finds all p tags with an id of first.
- body p.outer-text — finds any p tags with a class of outer-text inside of a body tag.

In [25]:
soup.select("div p")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>,
 <p class="inner-text">
                 Second paragraph.
             </p>]

### 날씨 데이터 스크래핑

https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168
- 날씨 예보 웹 페이지 다운로드
- 웹 페이지 파싱하기 위한 BeautifulSoup 객체 생성
- id가 seven-dat-forecast인 div 태그 찾고 이름 seven_day에 할당
- seven_day 내부에서 예보 아이템 찾기
- 첫 번째 아이템 추출 및 출력

In [26]:
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Tonight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Tonight: Partly cloudy, with a low around 48. Breezy, with a west northwest wind 17 to 22 mph becoming light west  after midnight. Winds could gust as high as 29 mph. " class="forecast-icon" src="DualImage.php?i=nwind_sct&amp;j=nsct" title="Tonight: Partly cloudy, with a low around 48. Breezy, with a west northwest wind 17 to 22 mph becoming light west  after midnight. Winds could gust as high as 29 mph. "/>
 </p>
 <p class="short-desc">
  Partly Cloudy
  <br/>
  and Breezy
  <br/>
  then Partly
  <br/>
  Cloudy
 </p>
 <p class="temp temp-low">
  Low: 48 °F
 </p>
</div>


- tonight 변수 안의 정보 추출
- 예보 아이템 이름: Tonight
- 예보 설명: img 태그 안의 title 속성
- 짧은 예보 설명: 위 경우에는 Mostly Clear임
- 최저 기온: 위 경우에는 47도

In [29]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)

Tonight
Partly Cloudyand Breezythen PartlyCloudy
Low: 48 °F
